In [1]:
import os
os.chdir('../')
os.getcwd()

'c:\\Users\\user\\Desktop\\DTI\\DeepPurpose-1'

In [2]:
from DeepPurpose import utils, dataset
from DeepPurpose import DTI as models
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

In [4]:
#### process_BindingDB ####

# Parameter
path = '../Database/BindingDB/BindingDB_All_202310_edit.tsv'
df = None
y = 'IC50'
binary = False
convert_to_log = True
threshold = 30
return_ids = False
ids_condition = 'OR'
harmonize_affinities = None

# loading df (bindingDB)
df = pd.read_csv(path, sep = '\t')

print('Beginning Processing...')
df = df[df['Number of Protein Chains in Target (>1 implies a multichain complex)'] == 1.0]
df = df[df['Ligand SMILES'].notnull()]

idx_str = []
yy = y
if isinstance(y, str):
	yy = [y]
for y in yy:
	if y == 'Kd':
		idx_str.append('Kd (nM)')
	elif y == 'IC50':
		idx_str.append('IC50 (nM)')
	elif y == 'Ki':
		idx_str.append('Ki (nM)')
	elif y == 'EC50':
		idx_str.append('EC50 (nM)')
	else:
		print('select Kd, Ki, IC50 or EC50')

if len(idx_str)==1:
	df_want = df[df[idx_str[0]].notnull()]
else: # select multiple affinity measurements.                 
	# keep rows for which at least one of the columns in the idx_str list is not null
	df_want = df.dropna(thresh=1, subset=idx_str) 
	
df_want = df_want[['BindingDB Reactant_set_id', 'Ligand InChI', 'Ligand SMILES',\
				'PubChem CID', 'UniProt (SwissProt) Primary ID of Target Chain',\
				'BindingDB Target Chain Sequence'] + idx_str]

for y in idx_str:
	df_want[y] = df_want[y].str.replace('>', '')
	df_want[y] = df_want[y].str.replace('<', '')
	df_want[y] = df_want[y].astype(float)

# Harmonize into single label using the mean of existing labels:
df_want['Label'] = df_want[idx_str].mean(axis=1, skipna=True)

df_want.rename(columns={'BindingDB Reactant_set_id':'ID',
						'Ligand SMILES':'SMILES',
						'Ligand InChI':'InChI',
						'PubChem CID':'PubChem_ID',
						'UniProt (SwissProt) Primary ID of Target Chain':'UniProt_ID',
						'BindingDB Target Chain Sequence': 'Target Sequence'},
						inplace=True)

# have at least uniprot or pubchem ID
if ids_condition == 'OR':
	df_want = df_want[df_want.PubChem_ID.notnull() | df_want.UniProt_ID.notnull()]
elif ids_condition == 'AND':
	df_want = df_want[df_want.PubChem_ID.notnull() & df_want.UniProt_ID.notnull()]
else:
	ValueError("ids_condition must be set to 'OR' or 'AND'")

df_want = df_want[df_want.InChI.notnull()]

df_want = df_want[df_want.Label <= 10000000.0]
print('There are ' + str(len(df_want)) + ' drug target pairs.')

if harmonize_affinities is not None:
	df_want = df_want[['PubChem_ID', 'SMILES', 'UniProt_ID', 'Target Sequence', 'Label']]
	if harmonize_affinities.lower() == 'max_affinity':
		df_want = df_want.groupby(['PubChem_ID', 'SMILES', 'UniProt_ID', 'Target Sequence']).Label.agg(min).reset_index()
	if harmonize_affinities.lower() == 'mean':
		df_want = df_want.groupby(['PubChem_ID', 'SMILES', 'UniProt_ID', 'Target Sequence']).Label.agg(np.mean).reset_index()

if binary:
	print('Default binary threshold for the binding affinity scores are 30, you can adjust it by using the "threshold" parameter')
	if isinstance(threshold, Sequence):
		# filter samples with affinity values between the thresholds
		df_want = df_want[(df_want.Label < threshold[0]) | (df_want.Label > threshold[1])]
	else: # single threshold
		threshold = [threshold]
	y = [1 if i else 0 for i in df_want.Label.values < threshold[0]]
else:
	if convert_to_log:
		print('Default set to logspace (nM -> p) for easier regression')
		y = utils.convert_y_unit(df_want.Label.values, 'nM', 'p')
	else:
		y = df_want.Label.values
	
if return_ids:
	X_drug, X_target, y, ID_drug, ID_target = df_want.SMILES.values, df_want['Target Sequence'].values, np.array(y), df_want['PubChem_ID'].values, df_want['UniProt_ID'].values
X_drug, X_target, y = df_want.SMILES.values, df_want['Target Sequence'].values, np.array(y)


Beginning Processing...
There are 1592671 drug target pairs.
Default set to logspace (nM -> p) for easier regression


In [5]:
#### select endcoder ####
drug_encoding, target_encoding = 'CNN', 'CNN'

In [6]:
#### data_process ####

# parameter
split_method = 'random'
frac = [0.7, 0.1, 0.2]
random_seed = 1
sample_frac = 1
mode = 'DTI'
X_drug_ = None
X_target_ = None

# data_process : dataframe

if random_seed == 'TDC':
	random_seed = 1234
 
#property_prediction_flag = X_target is None
property_prediction_flag, function_prediction_flag, DDI_flag, PPI_flag, DTI_flag = False, False, False, False, False

if (X_drug is not None) and (X_target is not None):
	DTI_flag = True
	if (X_drug is None) or (X_target is None):
		raise AttributeError("Target pair sequence should be in X_target, X_drug")
else:
	raise AttributeError("Please use the correct mode. Currently, we support DTI, DDI, PPI, Drug Property Prediction and Protein Function Prediction...")

if split_method == 'repurposing_VS':
	y = [-1]*len(X_drug) # create temp y for compatitibility

if DTI_flag:
	print('Drug Target Interaction Prediction Mode...')
	if isinstance(X_target, str):
		X_target = [X_target]
	if len(X_target) == 1:
		# one target high throughput screening setting
		X_target = np.tile(X_target, (length_func(X_drug), ))

	df_data = pd.DataFrame(zip(X_drug, X_target, y))
	df_data.rename(columns={0:'SMILES',
							1: 'Target Sequence',
							2: 'Label'}, 
							inplace=True)
	print('in total: ' + str(len(df_data)) + ' drug-target pairs')
 
# data_process : endcoding & splitting

if sample_frac != 1:
	df_data = df_data.sample(frac = sample_frac) #.reset_index(drop = True)
	print('after subsample: ' + str(len(df_data)) + ' data points...') 

if DTI_flag:
	df_data = utils.encode_drug(df_data, drug_encoding)
	df_data = utils.encode_protein(df_data, target_encoding)

Drug Target Interaction Prediction Mode...
in total: 1592671 drug-target pairs
encoding drug...
unique drugs: 770912
encoding protein...
unique target sequence: 5735


In [7]:
# dti split
if DTI_flag:
	if split_method == 'repurposing_VS':
		pass
	else:
		print('splitting dataset...')

	if split_method == 'random': 
		train, val, test = utils.create_fold(df_data, random_seed, frac)
	elif split_method == 'cold_drug':
		train, val, test = utils.create_fold_setting_cold_drug(df_data, random_seed, frac)
	elif split_method == 'HTS':
		train, val, test = utils.create_fold_setting_cold_drug(df_data, random_seed, frac)
		val = pd.concat([val[val.Label == 1].drop_duplicates(subset = 'SMILES'), val[val.Label == 0]])
		test = pd.concat([test[test.Label == 1].drop_duplicates(subset = 'SMILES'), test[test.Label == 0]])        
	elif split_method == 'cold_protein':
		train, val, test = utils.create_fold_setting_cold_protein(df_data, random_seed, frac)
	elif split_method == 'repurposing_VS':
		train = df_data
		val = df_data
		test = df_data
	elif split_method == 'no_split':
		print('do not do train/test split on the data for already splitted data')
		results = df_data.reset_index(drop=True)
	else:
		raise AttributeError("Please select one of the three split method: random, cold_drug, cold_target!")

print('Done.')

splitting dataset...
Done.


In [20]:
#### save & reset index ####
pd.Series(train.index.values, name='index_train').to_csv('C:\\Users\\user\\Desktop\\DTI\\Database\\BindingDB\\BindingDB_index_train.tsv', sep='\t', index=True)
pd.Series(val.index.values, name='index_val').to_csv('C:\\Users\\user\\Desktop\\DTI\\Database\\BindingDB\\BindingDB_index_val.tsv', sep='\t', index=True)
pd.Series(test.index.values, name='index_test').to_csv('C:\\Users\\user\\Desktop\\DTI\\Database\\BindingDB\\BindingDB_index_test.tsv', sep='\t', index=True)

train, val, test = train.reset_index(drop=True), val.reset_index(drop=True), test.reset_index(drop=True)

In [19]:
pd.Series(val.index.values, name='index_val').to_csv('C:\\Users\\user\\Desktop\\DTI\\Database\\BindingDB\\BindingDB_index_val.tsv', sep='\t', index=True)

In [21]:
# Model configuration genreration
config = utils.generate_config(drug_encoding = drug_encoding, 
                         target_encoding = target_encoding, 
                         cls_hidden_dims = [1024,1024,512], 
                         train_epoch = 50, 
                         test_every_X_epoch = 10, 
                         LR = 0.001, 
                         batch_size = 256,
                         hidden_dim_drug = 128,
                         cnn_drug_filters = [32,64,96],
                         cnn_drug_kernels = [4,6,8],
                         cnn_target_filters = [32,64,96],
                         cnn_target_kernels = [4,8,12]
                        )

In [22]:
# Model initialization
model = models.model_initialize(**config)
model

In [23]:
# Model training
model.train(train, val, test)

# Model saving
model.save_model('./pretrained_model/CNN_CNN_KIBA_after/')

Let's use 1 GPU!
--- Data Preparation ---
--- Go for Training ---
Training at Epoch 1 iteration 0 with loss 41.8660. Total time 0.00083 hours
Training at Epoch 1 iteration 100 with loss 2.08345. Total time 0.02055 hours
Training at Epoch 1 iteration 200 with loss 2.37071. Total time 0.04 hours
Training at Epoch 1 iteration 300 with loss 1.93328. Total time 0.05888 hours
Training at Epoch 1 iteration 400 with loss 1.71777. Total time 0.07833 hours
Training at Epoch 1 iteration 500 with loss 1.63799. Total time 0.0975 hours
Training at Epoch 1 iteration 600 with loss 1.60238. Total time 0.11666 hours
Training at Epoch 1 iteration 700 with loss 1.72801. Total time 0.13611 hours
Training at Epoch 1 iteration 800 with loss 1.65600. Total time 0.15527 hours
Training at Epoch 1 iteration 900 with loss 1.44310. Total time 0.17444 hours
Training at Epoch 1 iteration 1000 with loss 1.45975. Total time 0.19388 hours
Training at Epoch 1 iteration 1100 with loss 1.62839. Total time 0.21305 hours
Tr

KeyboardInterrupt: 